In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

#运行次数
max_steps=1001

#图片数量
image_num=3000

#文件路径
DIR='/home/x/deeplearning/Tenworflow/Learn_TF/'

#载入图片
embedding=tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) #平均值
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)  #标准差
        tf.summary.scalar('max',tf.reduce_max(var))  #最大值
        tf.summary.scalar('min',tf.reduce_min(var))  #最小值
        tf.summary.histogram('histogram',var)  #直方图
        

#初始化权值
def weight_variable(shape,name):
    initial=tf.truncated_normal(shape,stddev=0.1,name=name)#生成一个截断的正态分布
    return tf.Variable(initial)

#初始化偏置
def biases_variable(shape,name):
    initial=tf.constant(0.1,shape=shape,name=name)
    return tf.Variable(initial)

#卷积层
def conv2d(x,W):
    # x input tensor of shape '[batch,in_height,in_width,in_channels]'
    #W filter / kernel tensor of shape [filter_height,filter_width,in_channels,out_channels]
    # strides[0]=strides[3]=1.  stride[1]代表x方向的步长,stride[2]代表y方向的步长.
    # padding: A 'string' from: 'SAME','VALID'
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize[1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#定义两个placeholder
with tf.name_scope('input'):
    x=tf.placeholder(tf.float32,[None,784],name='x-input')#28*28
    y=tf.placeholder(tf.float32,[None,10],name='y-input')

#显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input=tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shaped_input,10)    
    
    
#改变x的格式转为4D的向量'[batch,in_height,in_width,in_channels]'
with tf.name_scope('input_x_image'):
    x_image=tf.reshape(x,[-1,28,28,1])
    
#初始化第一个卷积层的权值和偏置
with tf.name_scope('W_conv1'):
    W_conv1=weight_variable([5,5,1,32],'W_conv1')#32个5x5卷积核从1个平面卷积
with tf.name_scope('b_conv1'):
    b_conv1=biases_variable([32],'b_conv1')#每一个卷积核一个偏置值

#把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
with tf.name_scope('output_conv1'):
    h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
with tf.name_scope('output_pooling1'):
    h_pool1=max_pool_2x2(h_conv1)#进行max_pooling

#初始化第二个卷积层的权值和偏置
with tf.name_scope('W_conv2'):
    W_conv2=weight_variable([5,5,32,64],'W_conv2')#64个5x5卷积核从32个平面卷积
with tf.name_scope('b_conv1'):
    b_conv2=biases_variable([64],'b_conv2')#每一个卷积核一个偏置值

#把h_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
with tf.name_scope('output_conv2'):
    h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
with tf.name_scope('output_pooling2'):
    h_pool2=max_pool_2x2(h_conv2)#进行max_pooling

#28×28的图片第一次卷积后还是32张28*28（SAME方式padding）,第一次pooling后32张14*14
#第二次卷积后为64张14*14，第二次pooling后变为64张7*7
#最后为64张7*7平面

#初始化第一个全连接层的权值
with tf.name_scope('W_fc1'):
    W_fc1=weight_variable([7*7*64,1024],'W_fc1')
with tf.name_scope('b_fc1'):
    b_fc1=biases_variable([1024],'b_fc1')

#把池化层2的输出扁平化为1维
with tf.name_scope('output_pooling2_flat'):
    h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
#求第一个全连接层的输出
with tf.name_scope('output_fc1'):
    h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

#keep_prob用来表示神经元dropout的输出概率
with tf.name_scope('keep_prob'):
    keep_prob=tf.placeholder(tf.float32,name='keep_prob')
with tf.name_scope('output_fc1_drop_out'):
    h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

#初始化第二个全连接层的权值
with tf.name_scope('W_fc2'):
    W_fc2=weight_variable([1024,10],'W_fc2')
    variable_summaries(W_fc2)
with tf.name_scope('b_fc2'):
    b_fc2=biases_variable([10],'b_fc2')

#求第二个全连接层的输出
with tf.name_scope('softmax'):
    prediction=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

#交叉熵为代价函数
with tf.name_scope('cross_entropy'):
    cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('cross_entropy',cross_entropy)
#使用Adam进行优化
with tf.name_scope('train'):
    train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔列表中
with tf.name_scope('accuracy'):
    with tf.name_scope('corrent_prediction'):
        correct_prediction =tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
#求准确率
    with tf.name_scope('accuracy'):
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #产生metdata文件
    if tf.gfile.Exists(DIR+'projector2/projector/metadata.tsv'):
        tf.gfile.DeleteRecursively(DIR+'projector2/projector/metadata.tsv')
    with open (DIR+'projector2/projector/metadata.tsv','w') as f:
        labels=sess.run(tf.argmax(mnist.test.labels[:],1))
        for i in range(image_num):
            f.write(str(labels[i])+'\n')
    
    #合并所有summary
    merged=tf.summary.merge_all()
    
    projector_writer=tf.summary.FileWriter(DIR+'projector2/projector',sess.graph)
    saver=tf.train.Saver()
    config=projector.ProjectorConfig()
    embed=config.embeddings.add()
    embed.tensor_name=embedding.name
    embed.metadata_path=DIR+'projector2/projector/metadata.tsv'
    embed.sprite.image_path=DIR+'projector2/data/mnist_10k_sprite.png'
    embed.sprite.single_image_dim.extend([28,28])
    projector.visualize_embeddings(projector_writer,config)
    
    for epoch in range(max_steps):
        batch_xs,batch_ys=mnist.train.next_batch(100)
        run_options=tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata=tf.RunMetadata()
        summary,_=sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7},options=run_options,run_metadata=run_metadata)
        projector_writer.add_run_metadata(run_metadata,'step%03d' % epoch)
        projector_writer.add_summary(summary,epoch)

        
        
        if epoch%100==0:
            testSet = mnist.test.next_batch(5000)
            acc=sess.run(accuracy,feed_dict={x:testSet[0],y:testSet[1],keep_prob:1.0}) 
            print('Iter'+str(epoch) +' : Testing Accuracy:'+str(acc))
            #print("test accuracy %g"%accuracy.eval(feed_dict={ x: testSet[0], y: testSet[1], keep_prob: 1.0}))  
            
    saver.save(sess,DIR+'projector2/projector/a_model.ckpt',global_step=max_steps)
    projector_writer.close()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Iter0 : Testing Accuracy:0.1348
Iter100 : Testing Accuracy:0.7324
Iter200 : Testing Accuracy:0.8306
Iter300 :